In [ ]:
import os
import itertools
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
from simulator_utilities import *
np.random.seed(7)

In [ ]:
# ========================
# Preparation
# ========================
# Output
folder_path = r'/w/repos/slate_sim/data/no_changes'
summary_file_path = os.path.join(folder_path, 'simulation_data_summary.csv')
context_file_path = os.path.join(folder_path, 'simulation_data_{0}.csv')
all_file_path = os.path.join(folder_path, 'simulation_data2.csv')

# Contexts 
contexts = {'platform': ['Mac', 'Windows'], 
            'network': ['wifi', 'wired'], 
            'country': ['US', 'CA']}
unique_contexts = [list(x) for x in itertools.product(*contexts.values())]

# Parameter statistics
params = {
          'x': {'mean': 1, 'min': 0, 'max': 4, 'std_range': [0.1, 1.1]},
          'y': {'mean': 1, 'min': 0, 'max': 3, 'std_range': [0.1, 1.1]}, 
          'z': {'mean': 1, 'min': 0, 'max': 2, 'std_range': [0.1, 1.1]}
         }

# Initialization
data = []
dist_context = {}
reward_range = [0.05, 0.35]
coefficient_range = [0.1, 1]
interaction2 = True
known_n = None
descritization_policy = {'x': 8, 'y': 6, 'z': 4}
share_descritized_grid = True

# Confidence Interval statistics
ci_mean = 0
ci_std = 0.1
ci_dist = gen_distribution('normal', ci_mean, ci_std, 5000)[0]

# N for each unique context
if known_n:
    n_per_config = known_n
else:
    ci_width = 0.005
    ci_mult = 1.96
    n_per_config = int(((ci_mult*ci_std/(ci_width/2))**2//100+1)*100)
n_per_context = np.prod([x if isinstance(x, int) else len(x)for x in descritization_policy.values()]) * n_per_config
print('Data Size per Configuration: {:,}'.format(n_per_config))
print('Data Size per Context: {:,}'.format(n_per_context))
print('Total Data Size: {:,}'.format(n_per_context*len(unique_contexts)))

In [ ]:
# ========================
# Generate data
# ========================
param_list = list(params.keys())
df_cols = list(contexts.keys()) + param_list + ['reward']
plot_pairs = [x for x in itertools.combinations(range(len(param_list)), 2)]
df_summary = pd.DataFrame()

df_all = pd.DataFrame()

if share_descritized_grid:
    for k, v in descritization_policy.items():
        if isinstance(v, int):
            descritization_policy[k] = [round(x,4) for x in np.linspace(params[k]['min'], params[k]['max'], v)]

for i, c in enumerate(unique_contexts):
    
#     display(Markdown('**[{0}/{1}] Generating data for context {2} ...**'.format(i+1, len(unique_contexts), c)))

    # [1] Generate Distributions
    n_dist = 5000
    dist_context[i] = gen_param_reward(params, reward_range, n_dist)
#     plot_1d_param_reward(dist_context[i])
    
    # [2] Coefficients
    if interaction2:
        inter_terms = [x for x in itertools.combinations(range(len(param_list)), 2)]
    else:
        inter_terms = []
    n_coef = len(param_list) + len(inter_terms)
    coefficients = np.random.uniform(coefficient_range[0], coefficient_range[1], n_coef)    
    
    # [3] Discretize parameter
    discretize_parameters(dist_context[i], descritization_policy, equal_distance=True)
    dist_context[i]['configs'] = gen_config_reward(dist_context[i], param_list)
    
    # [4] Reward formula
    reward_formula = param_list + ['{0}{1}'.format(param_list[x[0]], param_list[x[1]]) for x in inter_terms]
    dist_context[i]['configs']['reward_equation'] = formulate_equation(reward_formula, coefficients)
    dist_context[i]['configs']['config_rterms'] = add_interactions(dist_context[i]['configs']['config_rterms'], inter_terms)
    display(Markdown('* {0}'.format(dist_context[i]['configs']['reward_equation'])))
    
    # [5] Generate data
    num_values = gen_data(dist_context[i], n_per_config, ci_dist, coefficients, reward_range)
    dist_context[i]['configs']['config_reward'] = np.array([x[-1] for x in num_values])
    c_data = [c + list(x) for x in num_values]
    data = data + c_data
    
    # [6] Plot 2D
#     plot_data = pd.DataFrame(num_values, columns=param_list + ['reward'])
#     plot_2d_paris(plot_data, param_list, inter_terms, round_to=0.05)
            
    # [7] To file
    df_context = pd.DataFrame(c_data, columns=df_cols)
    df_context = df_context.sample(frac=1)
    df_all = df_all.append(df_context)
    
    # [8] Summary
    df_mean = summarize_df(df_context, param_list, c)
    df_summary = df_summary.append(df_mean)

In [ ]:
# Save summary file
df_summary.to_csv(summary_file_path, index=False)

In [ ]:
all_file_path = os.path.join(folder_path, 'simulation_data_scaledto1.csv')
df_all.to_csv(all_file_path, index=False)

In [ ]:
pd.DataFrame(df_all["reward"]).query('reward>0.35').sort_values("reward")